## Amazon Bedrock Knowledge Bases를 사용한 Q&A 애플리케이션 구축 - Retrieve API

### 컨텍스트
이 노트북에서는 Amazon Bedrock Knowledge Bases의 Retrieve API를 사용하여 Q&A 애플리케이션을 구축하는 방법을 자세히 살펴보겠습니다. 여기서는 유사도 검색을 기반으로 원하는 수의 문서 청크를 얻기 위해 지식 베이스에 쿼리를 수행합니다. 그런 다음 관련 문서와 쿼리로 프롬프트를 보강하여 Anthropic Claude V2에 입력으로 전달하여 응답을 생성합니다.

지식 베이스를 사용하면 회사 데이터에 Amazon Bedrock의 기초 모델(FM)을 안전하게 연결하여 검색 증강 생성(RAG)을 수행할 수 있습니다. 추가 데이터에 대한 접근은 FM을 지속적으로 재훈련하지 않고도 모델이 더 관련성 있고, 컨텍스트에 특화되며, 정확한 응답을 생성하는 데 도움이 됩니다. 지식 베이스에서 검색된 모든 정보는 투명성을 높이고 환각을 최소화하기 위해 소스 출처와 함께 제공됩니다. 콘솔을 사용한 지식 베이스 생성에 대한 자세한 내용은 이 [게시물](https://docs.aws.amazon.com/bedrock/latest/userguide/knowledge-base.html)을 참조하세요.

노트북에서는 2가지 파트를 다룰 것입니다:

- 파트 1: Amazon Bedrock의 기초 모델과 함께 `RetrieveAPI`를 사용하는 방법을 공유합니다. `anthropic.claude-3-sonnet-20240229-v1:0` 모델을 사용할 것입니다.
- 파트 2: langchain 통합을 시연합니다.

### 패턴
Retrieval Augmented Generation(RAG) 패턴을 사용하여 솔루션을 구현할 수 있습니다. RAG는 언어 모델 외부에서 데이터를 검색하고(비매개변수적) 검색된 관련 데이터를 컨텍스트에 추가하여 프롬프트를 보강합니다. 여기서는 콘솔/SDK를 사용하여 생성된 Knowledge Base에서 효과적으로 RAG를 수행하고 있습니다.

### 사전 요구사항
질문에 답변하기 전에 문서들을 처리하여 지식 베이스에 저장해야 합니다. 이 노트북에서는 Amazon Bedrock Knowledge Bases를 생성하기 위해 `synthetic dataset for 10K financial reports`을 사용합니다.

문서(데이터 소스)를 Amazon S3 버킷에 업로드합니다.
[01_create_ingest_documents_test_kb_multi_ds.ipynb](/knowledge-bases/01-rag-concepts/01_create_ingest_documents_test_kb_multi_ds.ipynb)를 사용하여 Amazon Bedrock Knowledge Bases를 생성합니다.
Knowledge Base ID를 기록해 둡니다.

### 노트북 설명
이 노트북에서는 Amazon Bedrock Knowledge Bases가 제공하는 `Retrieve API`를 사용합니다. 이 API는 사용자 쿼리를 임베딩으로 변환하고, 지식 베이스를 검색하여 관련 결과를 반환하여 의미론적 검색 결과를 기반으로 사용자 정의 워크플로우를 구축할 수 있는 더 많은 제어 기능을 제공합니다. `Retrieve API`의 출력에는 검색된 텍스트 청크, 소스 데이터의 위치 유형과 URI, 그리고 검색의 관련성 점수가 포함됩니다.

그런 다음 생성된 텍스트 청크를 사용하여 원본 프롬프트와 함께 보강하고, 사용 사례에 따른 프롬프트 엔지니어링 패턴을 사용하여 `anthropic.claude-3-sonnet-20240229-v1:0` 모델에 전달합니다.

### 사용 사례:

#### 데이터셋
이 예제에서는 Octank의 금융 10k 보고서(합성 생성 데이터셋)를 텍스트 코퍼스로 사용하여 Q&A를 수행합니다. 이 데이터는 이미 Amazon Bedrock Knowledge Bases에 수집되어 있습니다. 이 예제를 실행하려면 `knowledge base id`가 필요합니다.
특정 사용 사례에서는 다른 도메인 주제에 대한 다른 파일을 동기화하고 지식 베이스의 `retrieve API`를 사용하여 모델 응답을 평가하기 위해 동일한 방식으로 이 노트북에 쿼리할 수 있습니다.

#### Python 3.10
⚠ 이 실습을 위해서는 Python 3.10 런타임 기반으로 노트북을 실행해야 합니다. ⚠

Amazon SageMaker 스튜디오 외부의 로컬 환경에서 워크샵을 수행하는 경우 Python 런타임 > 3.10을 실행하고 있는지 확인하세요.

#### 설정
이 노트북을 실행하려면 다음 패키지들을 설치해야 합니다.

In [ ]:
%pip install --upgrade pip --quiet
%pip install -r ../requirements.txt --no-deps --quiet
%pip install -r ../requirements.txt --upgrade --quiet

#### Restart the kernel with the updated packages that are installed through the dependencies above

In [ ]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [ ]:
%store -r kb_id
# kb_id = "<knowledge base id>" If you have already created knowledge base, comment the `store -r kb_id` and provide knowledge base id here.

### Follow the steps below to initiate the bedrock client:
### Bedrock 클라이언트를 시작하기 위한 단계:
1. 필요한 라이브러리들을 임포트합니다. Bedrock 모델 선택을 위한 langchain, llm과 임베딩 모델 인스턴스를 포함하는 서비스 컨텍스트를 저장하기 위한 llama index를 함께 임포트합니다. 이 서비스 컨텍스트는 나중에 노트북에서 Q&A 애플리케이션의 응답을 평가하는 데 사용됩니다.
2. `retrieve API`를 통해 모든 텍스트 청크 검색을 얻은 후, 주어진 지식 베이스와 함께 RAG 패턴을 사용하여 쿼리 완성을 수행할 대규모 언어 모델로 `anthropic.claude-3-sonnet-20240229-v1:0`를 초기화합니다.

In [ ]:
import boto3
import pprint
from botocore.client import Config
import json

pp = pprint.PrettyPrinter(indent=2)
session = boto3.session.Session()
region = session.region_name
bedrock_config = Config(connect_timeout=120, read_timeout=120, retries={'max_attempts': 0})
bedrock_client = boto3.client('bedrock-runtime', region_name = region)
bedrock_agent_client = boto3.client("bedrock-agent-runtime",
                              config=bedrock_config, region_name = region)
print(region)

### 파트 1 - Amazon Bedrock의 기초 모델과 함께 Retrieve API 사용
Amazon Bedrock Knowledge Bases가 제공하는 `Retrieve API`를 호출하는 검색 함수를 정의합니다. 이 API는 사용자 쿼리를 임베딩으로 변환하고, 지식 베이스를 검색하여 관련 결과를 반환하며, 의미론적 검색 결과를 기반으로 사용자 정의 워크플로우를 구축할 수 있는 더 많은 제어 기능을 제공합니다. `Retrieve API`의 출력에는 검색된 텍스트 청크, 소스 데이터의 위치 유형과 URI, 그리고 검색의 관련성 점수가 포함됩니다. `retrievalConfiguration`에서 `overrideSearchType` 옵션을 사용하여 `HYBRID` 또는 `SEMANTIC` 중 하나를 선택할 수도 있습니다. 기본적으로는 가장 관련성 높은 결과를 제공하기 위해 적절한 전략을 선택하며, 하이브리드 또는 의미론적 검색을 사용하도록 기본 옵션을 재정의하려면 값을 `HYBRID/SEMANTIC`으로 설정할 수 있습니다.

<!-- ![retrieveAPI](./images/retrieveAPI.png) -->
<img src="./images/retrieveAPI.png" width=50% height=20% />

In [ ]:
def retrieve(query, kbId, numberOfResults=5):
    return bedrock_agent_client.retrieve(
        retrievalQuery= {
            'text': query
        },
        knowledgeBaseId=kbId,
        retrievalConfiguration= {
            'vectorSearchConfiguration': {
                'numberOfResults': numberOfResults,
                'overrideSearchType': "HYBRID", # optional
            }
        }
    )

#### 초기화된 LLM에서 응답을 쿼리하기 전에 Knowledge base id를 초기화하세요

다음으로, `retrieve API`를 호출하고 `knowledge base id`, `number of results`, `query`를 매개변수로 전달할 것입니다.

`score`: 반환된 각 텍스트 청크의 관련 점수를 볼 수 있습니다. 이 점수는 해당 텍스트가 쿼리와 얼마나 밀접하게 일치하는지를 나타내는 상관관계를 보여줍니다.

In [ ]:
query = "2024년 기준 인구가 높은 순으로 시도별 지역 목록을 설명 없이 번호가 매겨진 목록으로 제공해 주세요. 단 지역 명 옆에 주요 산업분야도 적어줘."

response = retrieve(query, kb_id, 5)
retrievalResults = response['retrievalResults']
pp.pprint(retrievalResults)

### retrieve API 응답에서 텍스트 청크 추출하기

아래 셀에서 검색 결과로부터 컨텍스트를 가져올 것입니다.

In [ ]:
# fetch context from the response
def get_contexts(retrievalResults):
    idx = 1
    contexts = []
    for retrievedResult in retrievalResults: 
        contexts.append('/n')
        contexts.append('/n')
        contexts.append('Retrieved_Data_'+str(idx))
        contexts.append(retrievedResult['content']['text'])
        idx = idx + 1
    return contexts

In [ ]:
contexts = get_contexts(retrievalResults)
pp.pprint(contexts)

### 응답을 개인화하기 위한 모델별 프롬프트
여기서는 아래의 특정 프롬프트를 사용하여 모델이 금융 자문 AI 시스템 역할을 하도록 할 것입니다. 이 시스템은 가능한 경우 사실 기반 및 통계 정보를 사용하여 질문에 답변을 제공할 것입니다. 위에서 얻은 `Retrieve API` 응답을 프롬프트의 `{contexts}` 부분으로 제공하여 모델이 참조할 수 있도록 하며, 사용자 `query`도 함께 제공할 것입니다.

In [ ]:
prompt = f"""
Human: 당신은 금융 자문 AI 시스템이며, 가능한 한 사실 기반 및 통계 정보를 사용하여 질문에 답변을 제공합니다.
다음 정보들을 사용하여 <question> 태그로 둘러싸인 질문에 간결한 답변을 제공하세요.
답을 모르는 경우, 모른다고 말하고 답을 지어내려고 하지 마세요.
<context>
{contexts}
</context>

<question>
{query}
</question>

응답은 구체적이어야 하며 가능한 한 통계나 숫자를 사용해야 합니다.

Assistant:"""

### Amazon Bedrock에서 기초 모델 호출하기
이 예제에서는 Amazon Bedrock의 `anthropic.claude-3-sonnet-20240229-v1:0` 기초 모델을 사용할 것입니다.
- 경쟁사 대비 낮은 가격으로 최대의 유용성을 제공하며, 대규모 AI 배포를 위한 신뢰할 수 있는 고성능 워크호스가 되도록 설계되었습니다. Claude 3 Sonnet은 이미지를 처리하여 텍스트 출력을 반환할 수 있으며, 200K 컨텍스트 윈도우를 제공합니다.
- 모델 특성
    - 이미지에서 텍스트 및 코드 생성, 다국어 대화, 복잡한 추론 및 분석 가능

In [ ]:
# payload with model paramters
messages=[{ "role":'user', "content":[{'type':'text','text': prompt.format(contexts, query)}]}]
sonnet_payload = json.dumps({
    "anthropic_version": "bedrock-2023-05-31",
    "max_tokens": 512,
    "messages": messages,
    "temperature": 0.5,
    "top_p": 1
        }  )

In [ ]:
modelId = 'anthropic.claude-3-haiku-20240307-v1:0' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'
response = bedrock_client.invoke_model(body=sonnet_payload, modelId=modelId, accept=accept, contentType=contentType)
response_body = json.loads(response.get('body').read())
response_text = response_body.get('content')[0]['text']

pp.pprint(response_text)

### 파트 2 - LangChain 통합
이 노트북에서는 Amazon Bedrock Knowledge Bases가 제공하는 Retrieve API와 LangChain을 사용하여 Q&A 애플리케이션을 구축하는 방법을 자세히 살펴볼 것입니다. 유사도 검색을 기반으로 원하는 수의 문서 청크를 얻기 위해 지식 베이스에 쿼리하고, 이를 LangChain retriever와 통합하여 질문 답변을 위해 `Anthropic Claude 3 Sonnet` 모델을 사용할 것입니다.

In [ ]:
# from langchain.llms.bedrock import Bedrock
import langchain
from langchain_aws import ChatBedrock
from langchain.retrievers.bedrock import AmazonKnowledgeBasesRetriever

llm = ChatBedrock(model_id=modelId, 
                  client=bedrock_client)

`AmazonKnowledgeBasesRetriever` 객체를 LangChain에서 생성합니다. 이 객체는 Amazon Bedrock Knowledge Bases가 제공하는 `Retreive API`를 호출하는데, 이는 사용자 쿼리를 임베딩으로 변환하고, 지식 베이스를 검색하여 관련 결과를 반환하며, 의미론적 검색 결과를 기반으로 사용자 정의 워크플로우를 구축할 수 있는 더 많은 제어 기능을 제공합니다. `Retreive API`의 출력에는 `retrieved text chunks`,`location type`과 `URI`, 그리고 검색의 관련성 `scores`가 포함됩니다.

In [ ]:
query = "2024년 시도별 인구이동 특징을 분석하여 설명하시오."
retriever = AmazonKnowledgeBasesRetriever(
        knowledge_base_id=kb_id,
        retrieval_config={"vectorSearchConfiguration": 
                          {"numberOfResults": 4,
                           'overrideSearchType': "SEMANTIC", # optional
                           }
                          },
        # endpoint_url=endpoint_url,
        # region_name=region,
        # credentials_profile_name="<profile_name>",
    )
docs = retriever.get_relevant_documents(
        query=query
    )
pp.pprint(docs)

## 응답을 개인화하기 위한 모델별 프롬프트
여기서는 아래의 특정 프롬프트를 사용하여 모델이 금융 자문 AI 시스템 역할을 하도록 할 것입니다. 이 시스템은 가능한 경우 사실 기반 및 통계 정보를 사용하여 질문에 답변을 제공할 것입니다. 위에서 얻은 Retrieve API 응답을 프롬프트의 `{context}` 부분으로 제공하여 모델이 참조할 수 있도록 하며, 사용자 `query`도 함께 제공할 것입니다.

In [ ]:
from langchain.prompts import PromptTemplate

PROMPT_TEMPLATE = """
Human: 당신은 금융 자문 AI 시스템이며, 가능한 한 사실 기반 및 통계 정보를 사용하여 질문에 답변을 제공합니다.
다음 정보들을 사용하여 <question> 태그로 둘러싸인 질문에 간결한 답변을 제공하세요.
답을 모르는 경우, 모른다고 말하고 답을 지어내려고 하지 마세요.
<context>
{context}
</context>

<question>
{question}
</question>

응답은 구체적이어야 하며 가능한 한 통계나 숫자를 사용해야 합니다.

Assistant:"""
claude_prompt = PromptTemplate(template=PROMPT_TEMPLATE, 
                               input_variables=["context","question"])

### 검색기와 위에서 정의한 LLM을 RetrievalQA Chain과 통합하여 Q&A 애플리케이션 구축하기

In [ ]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": claude_prompt}
)

In [ ]:
answer = qa.invoke(query)
pp.pprint(answer)

## 결론
`Retrieve API`를 사용하여 RAG 기반 애플리케이션을 커스터마이징할 수 있으며, Bedrock의 `InvokeModel API`를 사용하거나 `AmazonKnowledgeBaseRetriever`를 사용하여 LangChain과 통합할 수 있습니다.
`Retrieve API`는 Amazon Bedrock이 제공하는 모든 기초 모델을 사용할 수 있는 유연성을 제공하며, 사용 사례에 따라 HYBRID 또는 SEMANTIC 중 적절한 검색 유형을 선택할 수 있습니다. 하이브리드 검색 기능에 대한 자세한 내용은 이 [블로그](https://console.harmony.a2z.com/internal-ai-assistant?playground=#https://aws.amazon.com/blogs/machine-learning/knowledge-bases-for-amazon-bedrock-now-supports-hybrid-search/)를 참조하세요.

<div class="alert alert-block alert-warning">
<b>참고:</b> 불필요한 요금이 발생하지 않도록 KB(Knowledge Base), OSS 인덱스 및 관련 IAM 역할과 정책을 삭제하는 것을 잊지 마세요.
</div>